In [1]:
import duckdb as ddb

In [15]:
ddb.sql("INSTALL httpfs; LOAD httpfs")

In [17]:
con = ddb.connect("../air_quality.db")

In [18]:
con.execute("CREATE schema IF NOT EXISTS raw")

In [19]:
con.sql("""
        SET s3_access_key_id='';
        SET s3_secret_access_key='';
        SET s3_region='';
        """)

In [20]:
con.execute("""
            CREATE TABLE IF NOT EXISTS raw.air_quality_data(
                location_id BIGINT,
                sensors_id BIGINT,
                "location" VARCHAR,
                "datetime" TIMESTAMP,
                lat DOUBLE,
                lon DOUBLE,
                "parameter" VARCHAR,
                units VARCHAR,
                "value" DOUBLE,
                "month" VARCHAR,
                "year" BIGINT,
                ingestion_datetime TIMESTAMP
            )

    """)

In [23]:
con.execute("""
INSERT INTO raw.air_quality_data
SELECT 
    location_id, 
    sensors_id, 
    "location", 
    "datetime", 
    lat, 
    lon, 
    "parameter", 
    units, 
    "value",
    "month", 
    "year",
    current_timestamp AS ingestion_datetime
FROM read_csv('s3://openaq-data-archive/records/csv.gz/locationid=1365268/year=2024/month=01/*.csv.gz');
""")

In [24]:
con.close()